# **Modelo de Machine learning**

En esta sección, se desarrolla un modelo de **machine learning** con el objetivo de predecir la cantidad de **hurtos a personas** mensuales para los años **2024** y **2025**, utilizando como base los datos históricos de delitos en Barranquilla. El análisis se enfoca específicamente en los hurtos a personas, dado que representan una de las categorías de delitos más relevantes por su alta frecuencia e impacto significativo en la seguridad ciudadana. Para este fin, se empleará un modelo de series temporales **ARIMA** (Autoregressive Integrated Moving Average), ya que este tipo de modelo es especialmente adecuado para conjuntos de datos que se desarrollan a lo largo del tiempo, permitiendo capturar patrones históricos, tendencias y estacionalidad, proporcionando así predicciones precisas para los períodos futuros.

In [24]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

Importamos el conjunto de datos que obtuvimos en la sección anterior.

In [13]:
df_final = pd.read_csv(r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\df_final.csv')

Filtramos el conjunto de datos por el delito de "Hurtos a personas", que es el caso de interés en este caso.

In [ ]:
df_hurtos_personas = df_final[df_final['Delito'] == 'Hurtos a personas']
df_hurtos_personas.head()

,Fecha,Delito,Cantidad de casos
47,2019-01,Hurtos a personas,892
48,2019-02,Hurtos a personas,924
49,2019-03,Hurtos a personas,1164
50,2019-04,Hurtos a personas,760
51,2019-05,Hurtos a personas,959


In [17]:
df_hurtos_personas.shape

(57, 3)

Notamos que los datos que usaremos contienen **57** observaciones y **3** variables.

En este bloque de código, se asegura que la columna **"Fecha"** esté en un formato adecuado para análisis de series temporales. Se agrega el primer día del mes (**"-01"**) a cada valor en la columna **"Fecha"** para convertirlo en un formato estándar de fecha (**YYYY-MM-DD**) utilizando la función `pd.to_datetime` de **pandas**. Esto es esencial para trabajar con datos temporales de manera precisa, permitiendo ordenar, filtrar y analizar los datos por períodos específicos. Luego, se visualizan los primeros (**head()**) y últimos (**tail()**) registros del DataFrame para verificar que la transformación se haya realizado correctamente.

In [ ]:
# Asegurarnos de que la columna 'Fecha' esté en formato de fecha, añadiendo el primer día del mes
df_hurtos_personas['Fecha'] = pd.to_datetime(df_hurtos_personas['Fecha'] + '-01', format='%Y-%m-%d')
df_hurtos_personas.head()


,Fecha,Delito,Cantidad de casos
47,2019-01-01,Hurtos a personas,892
48,2019-02-01,Hurtos a personas,924
49,2019-03-01,Hurtos a personas,1164
50,2019-04-01,Hurtos a personas,760
51,2019-05-01,Hurtos a personas,959


In [19]:
df_hurtos_personas.tail()

,Fecha,Delito,Cantidad de casos
348,2023-07-01,Hurtos a personas,1707
349,2023-08-01,Hurtos a personas,1218
350,2023-09-01,Hurtos a personas,1026
351,2023-10-01,Hurtos a personas,180
352,2023-11-01,Hurtos a personas,72


En este bloque de código, se utiliza un modelo de series temporales **ARIMA** para predecir la cantidad de **hurtos a personas** para cada mes del año **2024**, basándose en los datos históricos disponibles. Primero, se filtran los datos por cada mes, ajustando un modelo **ARIMA** para cada uno, y luego se realiza la predicción para el siguiente mes en la serie temporal. Los resultados se almacenan en un DataFrame para cada mes y, finalmente, se consolidan todas las predicciones en un solo DataFrame. Este proceso se realiza de manera iterativa para generar las predicciones de todos los meses del año 2024. Posteriormente, el mismo procedimiento se aplica para el año **2025**, utilizando las predicciones de 2024 como base para continuar con la serie temporal. Esto asegura una predicción continua y permite obtener estimaciones para ambos años.

In [25]:
# Asegurarse de que la columna 'Fecha' esté en formato datetime
df_hurtos_personas['Fecha'] = pd.to_datetime(df_hurtos_personas['Fecha'], errors='coerce')

# Lista de meses para realizar las predicciones
meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11 ]

# Diccionario para almacenar los resultados de las predicciones
predicciones = []

# Realizar predicciones para cada mes
for mes in meses:
    # Filtrar los datos por mes
    df_mes = df_hurtos_personas[df_hurtos_personas['Fecha'].dt.month == mes]
    df_mes.set_index('Fecha', inplace=True)

    # Ajustar el modelo ARIMA (ajustar los parámetros p, d, q si es necesario)
    modelo_mes = ARIMA(df_mes['Cantidad de casos'], order=(1, 1, 1))  # p=1, d=1, q=1 como ejemplo
    modelo_fit_mes = modelo_mes.fit()

    # Realizar la predicción para el siguiente mes
    forecast_mes = modelo_fit_mes.forecast(steps=1)

    # Crear la fecha de la predicción
    forecast_date_mes = pd.to_datetime(f'2024-{mes:02d}-01')

    # Redondear la predicción al siguiente número entero
    forecast_mes_redondeado = round(forecast_mes[0])

    # Crear un DataFrame con las predicciones
    forecast_df_mes = pd.DataFrame({
        'Fecha': [forecast_date_mes],
        'Delito': ['Hurtos a personas'],  # Aquí puedes cambiar el delito según sea necesario
        'Cantidad de casos': [forecast_mes_redondeado]
    })

    # Almacenar la predicción
    predicciones.append(forecast_df_mes)

# Concatenar todas las predicciones en un solo DataFrame
df_predicciones = pd.concat(predicciones, ignore_index=True)

df_predicciones


,Fecha,Delito,Cantidad de casos
0,2024-01-01,Hurtos a personas,921
1,2024-02-01,Hurtos a personas,1974
2,2024-03-01,Hurtos a personas,800
3,2024-04-01,Hurtos a personas,1373
4,2024-05-01,Hurtos a personas,1192
5,2024-06-01,Hurtos a personas,1796
6,2024-07-01,Hurtos a personas,1776
7,2024-08-01,Hurtos a personas,1225
8,2024-09-01,Hurtos a personas,985
9,2024-10-01,Hurtos a personas,465


Se concatenan las predicciones generadas para el año **2024** con los datos históricos disponibles, formando un nuevo conjunto de datos completo que incluye tanto los valores observados como los predichos. Esto permite extender la serie temporal y utilizarla como base para realizar las predicciones de los meses del año **2025**, asegurando una continuidad en el análisis y maximizando la precisión de las estimaciones para el segundo año.

In [ ]:
df_modelo = pd.concat([df_hurtos_personas, df_predicciones], ignore_index=True)

In [ ]:
# Asegurarse de que la columna 'Fecha' esté en formato datetime
df_modelo['Fecha'] = pd.to_datetime(df_modelo['Fecha'], errors='coerce')

# Lista de meses para realizar las predicciones para 2025
meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # Meses de todo el año

# Diccionario para almacenar los resultados de las predicciones
predicciones_2025 = []

# Realizar predicciones para cada mes de 2025
for mes in meses:
    # Filtrar los datos por mes
    df_mes = df_modelo[df_modelo['Fecha'].dt.month == mes]
    df_mes.set_index('Fecha', inplace=True)

    # Ajustar el modelo ARIMA (ajustar los parámetros p, d, q si es necesario)
    modelo_mes = ARIMA(df_mes['Cantidad de casos'], order=(1, 1, 1))  # p=1, d=1, q=1 como ejemplo
    modelo_fit_mes = modelo_mes.fit()

    # Realizar la predicción para el siguiente mes
    forecast_mes = modelo_fit_mes.forecast(steps=1)

    # Crear la fecha de la predicción para el año 2025
    forecast_date_mes = pd.to_datetime(f'2025-{mes:02d}-01')

    # Redondear la predicción al siguiente número entero
    forecast_mes_redondeado = round(forecast_mes[0])

    # Crear un DataFrame con las predicciones
    forecast_df_mes = pd.DataFrame({
        'Fecha': [forecast_date_mes],
        'Delito': ['Hurtos a personas'],  # Aquí puedes cambiar el delito según sea necesario
        'Cantidad de casos': [forecast_mes_redondeado]
    })

    # Almacenar la predicción
    predicciones_2025.append(forecast_df_mes)

# Concatenar todas las predicciones de 2025 en un solo DataFrame
df_predicciones_2025 = pd.concat(predicciones_2025, ignore_index=True)

# Mostrar las predicciones de 2025
df_predicciones_2025


,Fecha,Delito,Cantidad de casos
0,2025-01-01,Hurtos a personas,921
1,2025-02-01,Hurtos a personas,1954
2,2025-03-01,Hurtos a personas,908
3,2025-04-01,Hurtos a personas,1373
4,2025-05-01,Hurtos a personas,172
5,2025-06-01,Hurtos a personas,1796
6,2025-07-01,Hurtos a personas,1768
7,2025-08-01,Hurtos a personas,1229
8,2025-09-01,Hurtos a personas,971
9,2025-10-01,Hurtos a personas,555


Finalmente, los datos históricos junto con las predicciones generadas para los años **2024** y **2025** se combinan en un único conjunto de datos. Este conjunto incluye tanto los registros originales como las estimaciones realizadas, proporcionando una visión completa de los hurtos a personas en el tiempo. Posteriormente, este conjunto de datos consolidado se exporta a un archivo **CSV**, facilitando su análisis y visualización en herramientas como **Power BI** para explorar tendencias y diseñar estrategias basadas en las predicciones futuras.

In [ ]:
df_modelos = pd.concat([df_modelo, df_predicciones_2025], ignore_index=True)
df_modelos.to_csv(r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\predicciones_hurtos_2024_2025.csv', index=False)